# Welcome to my Capstone Project Notebook for Applied Data Science

## Part 1: Set up Capstone Notebook (Week 1)

This notebook will be used for my experiments in github and python

         for the IBM-Coursera Applied Data Science capstone

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Display all rows of a dataset
pd.options.display.max_rows = 1000

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Part 2: Segmenting and Clustering Neighborhoods in Toronto (Week 3)

### Read first table into dataframe

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Read html
dfs = pd.read_html(url,match='M[1-9].+')

# This also fetches a second unwanted table from lower down so we only want the 1st element:
dfT = dfs[0]
dfT.head()

Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

### Step 3 of instructions - Dataframe structure and contents

Assumptions: None. Current version of Wiki page is very straight forward.

Updates to instructions: See comments in next box

In [5]:
# 1. Rename first column to remove blank

column_names = ["PostalCode", "Borough", "Neighborhood"]
dfT.columns = column_names

# 2. Remove rows where Borough is 'Not assigned'
dfT = dfT.drop(dfT[dfT.Borough=='Not assigned'].index)

# 3. More than one neighborhood can exist in one postal code area.
#    These two rows will be combined into one row with the neighborhoods separated with a comma
#      -> No longer needed. Wiki used as input has been cleaned up
assert(dfT[dfT.duplicated(['PostalCode'])].shape[0] == 0)

# 4. If a cell has a borough but a Not assigned  neighborhood, 
#      then the neighborhood will be the same as the borough.
#      -> No longer needed. Wiki used as input has been cleaned up
assert(dfT[dfT.Neighborhood=='Not assigned'].shape[0] == 0)

# Display the start and end of the resulting table
dfT.head().append(dfT.tail())

PostalCode           Borough  \
2          M3A        North York   
3          M4A        North York   
4          M5A  Downtown Toronto   
5          M6A        North York   
6          M7A  Downtown Toronto   
160        M8X         Etobicoke   
165        M4Y  Downtown Toronto   
168        M7Y      East Toronto   
169        M8Y         Etobicoke   
178        M8Z         Etobicoke   

                                          Neighborhood  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
5                     Lawrence Manor, Lawrence Heights  
6          Queen's Park, Ontario Provincial Government  
160      The Kingsway, Montgomery Road, Old Mill North  
165                               Church and Wellesley  
168  Business reply mail Processing Centre, South C...  
169  Old Mill South, King's Mill Park, Sunnylea, Hu...  
178  Mimico NW, The Queensway West, South of Bloor,...

#### Shape of final DataFrame

In [6]:
dfT.shape

(103, 3)

### Determine the coordinates of each postcode

#### Note: I was unable to get google geocode to deliver reliable results => Switched to the Here API
This means that the coordinates are slightly different to google !
#### * Important note regarding Here API usage: This notebook is for private educational purposes only *

In [7]:
import requests

In [8]:
# @hidden_cell
API_Key = XXX

In [9]:
def get_coords_from_postcode(myAPI_Key,aPostCode):
    # Here API call for GET
    URL = "https://geocode.search.hereapi.com/v1/geocode?apiKey=" + myAPI_Key + "&qq=postalCode=" + aPostCode + ";country=Canada"
    
    # sending get request and saving the response as response object 
    r = requests.get(url = URL, params = {}) 
    
    # extracting data in json format 
    data = r.json()
    
    if (data['items'] == []):
        print ('No data returned for postcode "' + aPostCode + '"' )
        lat = 0
        long = 0
    else:
        lat  = data['items'][0]['position']['lat'] 
        long = data['items'][0]['position']['lng'] 

    return [lat,long]

In [10]:
dfT.head()

PostalCode           Borough                                 Neighborhood
2        M3A        North York                                    Parkwoods
3        M4A        North York                             Victoria Village
4        M5A  Downtown Toronto                    Regent Park, Harbourfront
5        M6A        North York             Lawrence Manor, Lawrence Heights
6        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

#### Add the columns 'Latitude' & 'Longitude' and set their values

In [11]:
dfT = dfT.reindex(columns=["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"])

In [12]:
for pc in dfT['PostalCode']:
    dfT.loc[dfT['PostalCode'] == pc, ['Latitude', 'Longitude']] = get_coords_from_postcode(API_Key,pc)    

No data returned for postcode "M7R"
No data returned for postcode "M5W"
No data returned for postcode "M7Y"


#### 3 postcodes are enclaves of others
It turns out that they are individual buildings so no one really lives there and it makes little sense to proces them separately

In [13]:
# Remove enclaves: At the time of code writing this removes M5W, M7R and M7Y
dfT = dfT[dfT.Latitude != 0]
dfT.shape

(100, 5)

#### This shape looks good: 2 additional columns and 3 fewer rows than before

In [14]:
dfT.head()

PostalCode           Borough                                 Neighborhood  \
2        M3A        North York                                    Parkwoods   
3        M4A        North York                             Victoria Village   
4        M5A  Downtown Toronto                    Regent Park, Harbourfront   
5        M6A        North York             Lawrence Manor, Lawrence Heights   
6        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

   Latitude  Longitude  
2  43.75244  -79.32927  
3  43.73042  -79.31334  
4  43.65512  -79.36264  
5  43.72327  -79.45160  
6  43.66107  -79.39089

### Explore and cluster the neighborhoods in Toronto
Lets make use of the New York code from earlier ...

In [15]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes   # Comment out after running once
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes   # Comment out after running once
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    _openmp_mutex-4.5          |            1_gnu          22 KB  conda-forge
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.12.5          |   py36h5fab9bb_0         143 KB  conda-forge
    openssl-1.1.1i             |       h7f98852_0         2.1 MB  conda-forge
    libgcc-ng-9.3.0            |      h5dbcf3e_17         7.8 MB  conda-forge
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    libgomp-9.3.0

#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user agent. We will name our agent tor_explorer, as shown below.

In [16]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfT['Latitude'], dfT['Longitude'], dfT['Borough'], dfT['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Folium is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods with 'Toronto' in the Borough name. So let's slice the original dataframe and create a new dataframe of the data.

In [22]:
toronto_data = dfT[dfT.Borough.str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

PostalCode           Borough                                 Neighborhood  \
0        M5A  Downtown Toronto                    Regent Park, Harbourfront   
1        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2        M5B  Downtown Toronto                     Garden District, Ryerson   
3        M5C  Downtown Toronto                               St. James Town   
4        M4E      East Toronto                                  The Beaches   

   Latitude  Longitude  
0  43.65512  -79.36264  
1  43.66107  -79.39089  
2  43.65736  -79.37818  
3  43.65121  -79.37548  
4  43.67653  -79.29541

We already have the geographical coordinates of Toronto.

In [23]:
# create map of Toronto using latitude and longitude values
map_toronto_inner = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_inner)  
    
map_toronto_inner

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [ ]:
# @hidden_cell

CLIENT_ID = YYY # your Foursquare ID
CLIENT_SECRET = ZZZ # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

## Explore Neighborhoods in Toronto

In [30]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            next
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                 latitudes=toronto_data['Latitude'],
                                 longitudes=toronto_data['Longitude']
                                )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

#### The size of the resulting dataframe¶

In [32]:
print(toronto_venues.shape)
toronto_venues.head()

(1456, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65512               -79.36264   
1  Regent Park, Harbourfront               43.65512               -79.36264   
2  Regent Park, Harbourfront               43.65512               -79.36264   
3  Regent Park, Harbourfront               43.65512               -79.36264   
4  Regent Park, Harbourfront               43.65512               -79.36264   

                    Venue  Venue Latitude  Venue Longitude  Venue Category  
0        Roselle Desserts       43.653447       -79.362017          Bakery  
1           Tandem Coffee       43.653559       -79.361809     Coffee Shop  
2  Figs Breakfast & Lunch       43.655675       -79.364503  Breakfast Spot  
3      Morning Glory Cafe       43.653947       -79.361149  Breakfast Spot  
4         Berkeley Church       43.655123       -79.365873     Event Space

#### How many venues were returned for each neighborhood

In [33]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            67   
Brockton, Parkdale Village, Exhibition Place                           55   
CN Tower, King and Spadina, Railway Lands, Harb...                     71   
Central Bay Street                                                     61   
Christie                                                               11   
Church and Wellesley                                                   80   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             29   
Davisville North                                                        7   
Dufferin, Dovercourt Village                                           16   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         2   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                       5   
High Park, The Junction South                                           3   
India Bazaar, The Beaches West                                         21   
Kensington Market, Chinatown, Grange Park                              48   
Lawrence Park                                                           2   
Little Portugal, Trinity                                               43   
Moore Park, Summerhill East                                             5   
North Toronto West, Lawrence Park                                       4   
Parkdale, Roncesvalles                                                 53   
Queen's Park, Ontario Provincial Government                            33   
Regent Park, Harbourfront                                              23   
Richmond, Adelaide, King                                              100   
Rosedale                                                                5   
Roselawn                                                                2   
Runnymede, Swansea                                                     41   
St. James Town                                                         90   
St. James Town, Cabbagetown                                            38   
Studio District                                                        50   
Summerhill West, Rathnelly, South Hill, Forest ...                      6   
The Annex, North Midtown, Yorkville                                    24   
The Beaches                                                             4   
The Danforth West, Riverdale                                            4   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         53   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             67   
Brockton, Parkdale Village, Exhibition Place                            55   
CN Tower, King and Spadina, Railway Lands, Harb...                      71   
Central Bay Street                                                      61   
Christie                                                                11   
Church and Wellesley                                                    80   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              29   
Davisville North                                                         7   
Dufferin, Dovercourt Village                                            16   
First Canadian Place, Underground city                                 100   
Forest Hill North & West, Forest H

#### How many unique categories can be curated from all the returned venues

In [34]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 212 unique categories.


## Analyze Each Neighborhood

In [35]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Adult Boutique  American Restaurant  Antique Shop  \
0            0               0                    0             0   
1            0               0                    0             0   
2            0               0                    0             0   
3            0               0                    0             0   
4            0               0                    0             0   

   Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  \
0            0           0                    0                 0   
1            0           0                    0                 0   
2            0           0                    0                 0   
3            0           0                    0                 0   
4            0           0                    0                 0   

   Athletics & Sports  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  \
0                   0          0           0           0       1     0    0   
1                   0          0           0           0       0     0    0   
2                   0          0           0           0       0     0    0   
3                   0          0           0           0       0     0    0   
4                   0          0           0           0       0     0    0   

   Baseball Field  Basketball Court  Basketball Stadium  Beer Bar  Beer Store  \
0               0                 0                   0         0           0   
1               0                 0                   0         0           0   
2               0                 0                   0         0           0   
3               0                 0                   0         0           0   
4               0                 0                   0         0           0   

   Belgian Restaurant  Bike Trail  Bistro  Board Shop  Bookstore  Boutique  \
0                   0           0       0           0          0         0   
1                   0           0       0           0          0         0   
2                   0           0       0           0          0         0   
3                   0           0       0           0          0         0   
4                   0           0       0           0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bubble Tea Shop  \
0                     0               0        0                0   
1                     0               0        0                0   
2                     0               1        0                0   
3                     0               1        0                0   
4                     0               0        0                0   

   Burger Joint  Burrito Place  Bus Line  Butcher  Café  Candy Store  \
0             0              0         0        0     0            0   
1             0              0         0        0     0            0   
2             0              0         0        0     0            0   
3             0              0         0        0     0            0   
4             0              0         0        0     0            0   

   Caribbean Restaurant  Cheese Shop  Chinese Restaurant  Church  \
0                     0            0                   0       0   
1                     0            0                   0       0   
2                     0            0                   0       0   
3                     0            0                   0       0   
4                     0            0                   0       0   

   Clothing Store  Cocktail Bar  Coffee Shop  College Arts Building  \
0               0             0            0                      0   
1               0             0            1                      0   
2               0             0            0                      0   
3               0             0            0                      0   
4               0             0            0                      0   

   College Auditorium  College Cafeteria  College Gym  Col

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                                         Berczy Park     0.014925   
1        Brockton, Parkdale Village, Exhibition Place     0.000000   
2   CN Tower, King and Spadina, Railway Lands, Har...     0.014085   
3                                  Central Bay Street     0.000000   
4                                            Christie     0.000000   
5                                Church and Wellesley     0.012500   
6                      Commerce Court, Victoria Hotel     0.010000   
7                                          Davisville     0.000000   
8                                    Davisville North     0.000000   
9                        Dufferin, Dovercourt Village     0.000000   
10             First Canadian Place, Underground city     0.000000   
11    Forest Hill North & West, Forest Hill Road Park     0.000000   
12                           Garden District, Ryerson     0.000000   
13  Harbourfront East, Union Station, Toronto Islands     0.000000   
14                      High Park, The Junction South     0.000000   
15                     India Bazaar, The Beaches West     0.000000   
16          Kensington Market, Chinatown, Grange Park     0.000000   
17                                      Lawrence Park     0.000000   
18                           Little Portugal, Trinity     0.023256   
19                        Moore Park, Summerhill East     0.000000   
20                  North Toronto West, Lawrence Park     0.000000   
21                             Parkdale, Roncesvalles     0.000000   
22        Queen's Park, Ontario Provincial Government     0.000000   
23                          Regent Park, Harbourfront     0.043478   
24                           Richmond, Adelaide, King     0.000000   
25                                           Rosedale     0.000000   
26                                           Roselawn     0.000000   
27                                 Runnymede, Swansea     0.000000   
28                                     St. James Town     0.000000   
29                        St. James Town, Cabbagetown     0.000000   
30                                    Studio District     0.000000   
31  Summerhill West, Rathnelly, South Hill, Forest...     0.000000   
32                The Annex, North Midtown, Yorkville     0.000000   
33                                        The Beaches     0.000000   
34                       The Danforth West, Riverdale     0.000000   
35           Toronto Dominion Centre, Design Exchange     0.000000   
36                     University of Toronto, Harbord     0.018868   

    Adult Boutique  American Restaurant  Antique Shop  Art Gallery  \
0           0.0000             0.000000          0.00     0.014925   
1           0.0000             0.000000          0.00     0.000000   
2           0.0000             0.000000          0.00     0.000000   
3           0.0000             0.000000          0.00     0.016393   
4           0.0000             0.000000          0.00     0.000000   
5           0.0125             0.012500          0.00     0.000000   
6           0.0000             0.040000          0.00     0.010000   
7           0.0000             0.000000          0.00     0.000000   
8           0.0000             0.000000          0.00     0.000000   
9           0.0000             0.000000          0.00     0.062500   
10          0.0000             0.030000          0.00     0.010000   
11          0.0000             0.000000          0.00     0.000000   
12          0.0000             0.000000          0.00     0.010000   
13          0.0000             0.000000          0.00     0.000000   
14          0.0000             0.000000          0.00     0.000000   
15          0.0000             0.000000          0.00     0.000000   
16          0.0000             0.000000          0.00     0.020833   
17          0.0000             0.000000          0.00     0.000000   
18          0.0000             0.023256          0

#### Print each neighborhood along with the top 5 most common venues¶

In [37]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.04
2  Seafood Restaurant  0.04
3            Beer Bar  0.03
4         Cheese Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0             Coffee Shop  0.11
1                    Café  0.07
2              Restaurant  0.07
3  Furniture / Home Store  0.05
4                     Bar  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.07
2                Café  0.04
3                Park  0.04
4                 Bar  0.04


----Central Bay Street----
             venue  freq
0      Coffee Shop  0.11
1   Clothing Store  0.07
2            Hotel  0.05
3            Plaza  0.03
4  Bubble Tea Shop  0.03


----Christie----
                venue  freq
0                Café  0.27
1       Grocery Store  0.1

Sort the venues in descending order

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Display the top 10 venues for each neighborhood

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place           Coffee Shop   
2  CN Tower, King and Spadina, Railway Lands, Har...           Coffee Shop   
3                                 Central Bay Street           Coffee Shop   
4                                           Christie                  Café   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar    Seafood Restaurant              Beer Bar   
1            Restaurant                  Café                   Bar   
2    Italian Restaurant                  Park                   Bar   
3        Clothing Store                 Hotel                 Plaza   
4         Grocery Store    Italian Restaurant           Candy Store   

       5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0             Farmers Market           Cheese Shop                 Hotel   
1     Furniture / Home Store             Gift Shop           Supermarket   
2                       Café  Gym / Fitness Center         Grocery Store   
3  Middle Eastern Restaurant        Cosmetics Shop            Restaurant   
4                 Playground    Athletics & Sports            Baby Store   

  8th Most Common Venue 9th Most Common Venue         10th Most Common Venue  
0                Lounge            Restaurant                         Bakery  
1    Italian Restaurant                Bakery  Vegetarian / Vegan Restaurant  
2     French Restaurant     Electronics Store                     Restaurant  
3        Sandwich Place       Bubble Tea Shop               Ramen Restaurant  
4           Coffee Shop    Falafel Restaurant                    Event Space

## Cluster Neighborhoods¶

Run k-means to cluster the neighborhood into 5 clusters

In [40]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

PostalCode           Borough                                 Neighborhood  \
0        M5A  Downtown Toronto                    Regent Park, Harbourfront   
1        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2        M5B  Downtown Toronto                     Garden District, Ryerson   
3        M5C  Downtown Toronto                               St. James Town   
4        M4E      East Toronto                                  The Beaches   

   Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.65512  -79.36264               1           Coffee Shop   
1  43.66107  -79.39089               1           Coffee Shop   
2  43.65736  -79.37818               1           Coffee Shop   
3  43.65121  -79.37548               1           Coffee Shop   
4  43.67653  -79.29541               1     Health Food Store   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0        Breakfast Spot       Thai Restaurant     Electronics Store   
1        Sandwich Place                  Café             Bookstore   
2        Clothing Store                  Café   Japanese Restaurant   
3                  Café    Seafood Restaurant    Italian Restaurant   
4                 Trail                   Pub           Wings Joint   

      5th Most Common Venue      6th Most Common Venue 7th Most Common Venue  \
0                Restaurant                        Spa                   Pub   
1  Mediterranean Restaurant         Italian Restaurant    College Auditorium   
2                     Hotel  Middle Eastern Restaurant        Cosmetics Shop   
3       American Restaurant               Cocktail Bar        Farmers Market   
4                   Dog Run                       Farm    Falafel Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0    Italian Restaurant            Food Truck    Distribution Center  
1          Intersection     College Cafeteria           College Quad  
2    Italian Restaurant         Movie Theater       Ramen Restaurant  
3           Cheese Shop                   Gym         Cosmetics Shop  
4           Event Space  Ethiopian Restaurant            Escape Room

Visualize the resulting clusters

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster

#### Cluster 1

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
21  Central Toronto               0                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
21      Basketball Court           Wings Joint            Donut Shop   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
21        Farmers Market                  Farm    Falafel Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
21           Event Space  Ethiopian Restaurant            Escape Room

#### Cluster 2

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0   Downtown Toronto               1           Coffee Shop   
1   Downtown Toronto               1           Coffee Shop   
2   Downtown Toronto               1           Coffee Shop   
3   Downtown Toronto               1           Coffee Shop   
4       East Toronto               1     Health Food Store   
5   Downtown Toronto               1           Coffee Shop   
6   Downtown Toronto               1           Coffee Shop   
7   Downtown Toronto               1                  Café   
8   Downtown Toronto               1           Coffee Shop   
9       West Toronto               1                  Park   
11      West Toronto               1                   Bar   
12      East Toronto               1         Grocery Store   
13  Downtown Toronto               1           Coffee Shop   
14      West Toronto               1           Coffee Shop   
15      East Toronto               1                  Park   
16  Downtown Toronto               1           Coffee Shop   
17      East Toronto               1                 Diner   
20   Central Toronto               1  Gym / Fitness Center   
24   Central Toronto               1                  Café   
25      West Toronto               1           Coffee Shop   
26   Central Toronto               1           Coffee Shop   
27  Downtown Toronto               1                  Café   
28      West Toronto               1           Coffee Shop   
29   Central Toronto               1     Convenience Store   
30  Downtown Toronto               1                  Café   
31   Central Toronto               1    Light Rail Station   
32  Downtown Toronto               1           Coffee Shop   
34  Downtown Toronto               1                  Café   
35  Downtown Toronto               1           Coffee Shop   
36  Downtown Toronto               1           Coffee Shop   

            2nd Most Common Venue        3rd Most Common Venue  \
0                  Breakfast Spot              Thai Restaurant   
1                  Sandwich Place                         Café   
2                  Clothing Store                         Café   
3                            Café           Seafood Restaurant   
4                           Trail                          Pub   
5                    Cocktail Bar           Seafood Restaurant   
6                  Clothing Store                        Hotel   
7                   Grocery Store           Italian Restaurant   
8                            Café                        Hotel   
9                   Grocery Store           Athletics & Sports   
11                   Cocktail Bar             Asian Restaurant   
12                           Park                     Bus Line   
13                           Café                        Hotel   
14                     Restaurant                         Café   
15           Fast Food Restaurant            Food & Drink Shop   
16                     Restaurant           Italian Restaurant   
17             Italian Restaurant                      Brewery   
20               Department Store            Food & Drink Shop   
24                 Sandwich Place   Modern European Restaurant   
25               Sushi Restaurant  Eastern European Restaurant   
26                           Park                         Café   
27                         Bakery                  Coffee Shop   
28                           Café                       Bakery   
29                     Restaurant                          Gym   
30  Vegetarian / Vegan Restaurant           Mexican Restaurant   
31                    Coffee Shop                  Supermarket   
32             Italian Restaurant                         Park   
34                    Coffee Shop                       Bakery   
35                          Hotel                   Restaurant   
36            Japanese Restaurant                   Restaurant   

   4th Most Common Venue      5th Most Comm

#### Cluster 3

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
18  Central Toronto               2              Bus Line   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
18           Swim School           Wings Joint            Donut Shop   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
18                  Farm    Falafel Restaurant           Event Space   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
18  Ethiopian Restaurant           Escape Room      Elementary School

#### Cluster 4

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
10  Downtown Toronto               3                  Park   
22      West Toronto               3     Convenience Store   
23   Central Toronto               3              Gym Pool   
33  Downtown Toronto               3                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
10       Harbor / Marina    Athletics & Sports           Music Venue   
22                  Park        Sandwich Place               Dog Run   
23                  Park            Playground                Garden   
33            Playground            Bike Trail   Japanese Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
10           Wings Joint               Dog Run                  Farm   
22    Falafel Restaurant           Event Space  Ethiopian Restaurant   
23   Distribution Center    Falafel Restaurant           Event Space   
33            Donut Shop                  Farm    Falafel Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
10    Falafel Restaurant           Event Space   Ethiopian Restaurant  
22           Escape Room     Elementary School      Electronics Store  
23  Ethiopian Restaurant           Escape Room      Elementary School  
33           Event Space  Ethiopian Restaurant            Escape Room

#### Cluster 5

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
19  Central Toronto               4        Clothing Store   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
19          Home Service               Dog Run        Farmers Market   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
19                  Farm    Falafel Restaurant           Event Space   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
19  Ethiopian Restaurant           Escape Room      Elementary School